In [1]:
import os

import gym
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd


from itertools import count
from collections import namedtuple
from collections import defaultdict
from torch.autograd import Variable
from tqdm import tnrange
from random import choice

from gridworld import GridworldEnv
from nets import *
from draw_methods import *
from hyper_parametrs import *
from env_methods import *
from tensorboardX import SummaryWriter
from utils import command

In [2]:
writer = SummaryWriter(logs_directory)
board = command.Command('tensorboard --logdir=run1:{} --port {}'.format(logs_directory, board_port))
board.run()

In [3]:
%pylab inline
%matplotlib inline

Populating the interactive namespace from numpy and matplotlib


/Users/andrejklimkin/anaconda/lib/python3.5/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['choice']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [4]:
env = GridworldEnv(shape=(5, 5))

In [5]:
state_dim = int(env.nS * 2)
action_dim = int(env.nA)

In [6]:
def entropy_term(probs):
    return -torch.sum(probs * torch.log(probs))

In [7]:
def update_params(agent, optimizer, losses_history):
    R = 0
    saved_actions = agent.saved_actions
    value_loss = 0
    rewards = []
    for r in agent.rewards[::-1]:
        R = r + gamma_rl * R
        rewards.insert(0, R)
    rewards = torch.Tensor(rewards)
    log_loss = Variable(torch.FloatTensor([0]))
    val_loss = Variable(torch.FloatTensor([0]))
    entropy_loss = Variable(torch.FloatTensor([0]))
    for (action, probs, value), r in zip(saved_actions, rewards):
        m = torch.distributions.Categorical(probs)
        reward = r - value.data[0, 0]
        log_loss  += -(m.log_prob(action[0]) * reward)
        val_loss += lambda_baseline * F.mse_loss(value, Variable(torch.Tensor([r])))
        entropy_loss += -entropy_weights["agent"] * entropy_term(probs)
    
    losses_history["entropy"].append(entropy_loss.data.numpy()[0])
    losses_history["value loss"].append(val_loss.data.numpy()[0])
    losses_history["log loss"].append(log_loss.data.numpy()[0])
    loss = log_loss + val_loss + entropy_loss
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    del agent.rewards[:]
    del agent.saved_actions[:]


In [8]:
Expert_left_corner = Expert("left_top_corner")
Expert_left_corner.precompute_v_function(env)
Expert_right_corner = Expert("right_bottom_corner")
Expert_right_corner.precompute_v_function(env)
Expert_global_optimal = Expert("global_optimal")
Expert_global_optimal.precompute_v_function(env)
possible_s_stars = [0, env.nS - 1]

In [9]:
def goal_unsim(env, g1, g2):
    return abs(g1 % int(np.sqrt(env.nS)) - g2 % int(np.sqrt(env.nS))) + abs(g1 // int(np.sqrt(env.nS)) - g2 // int(np.sqrt(env.nS)))

def goal_based_training(env, tmax, agent, goal, optimizer, losses_history, agent_type): 
    time = 0
    done = False
    while True:
        if is_terminal(env, env.s): #done
            break
        time += 1
        a = agent.select_action(env.s, goal)
        state, _, done, _= env.step(a[0, 0])
        if is_terminal(env, env.s):
            if is_terminal(env, env.s): #done
                break
        if time >= tmax:
            break
        agent.rewards.append(0)
    
    if agent_type == "goal-based":
        final_reward = (-time - lambda_goals  * goal_unsim(env, goal, env.s)) / scale_reward 
    else:
        final_reward = (-time) / scale_reward 
    agent.rewards.append(final_reward)
    if(len(agent.saved_actions)):
        update_params(agent, optimizer, losses_history)
    
    del agent.rewards[:]
    del agent.saved_actions[:]

    return final_reward
    

In [10]:
def choose_goal(env, s0, agent_type="goal-based"):
        if agent_type == "goal-based":
            if (s0 // np.sqrt(env.nS) <= int(np.sqrt(env.nS)) - (s0 % int(np.sqrt(env.nS))) - 1):
                return 0
            if (s0 // np.sqrt(env.nS) > int(np.sqrt(env.nS)) - (s0 % int(np.sqrt(env.nS))) - 1):
                return env.nS - 1
            #else:
            #    return np.random.choice([0, env.nS - 1])
        else:
            return None
    
def train_agent(env, tmax, each_goal_times, agent, optimizer, losses_history, agent_type):
    rewards = []
    episode_time = []
    finish_states = []
    set_random_s0(env)
    s0 = env.reset()
    goal = choose_goal(env, s0, agent_type)
    res = goal_based_training(env, tmax, agent, goal, optimizer, losses_history, agent_type)   
    rewards.append(res)
    
    return agent, np.array(rewards)

In [11]:
def play_n_episodes(n, env, model, s_star, tmax=40):
    state_dist_list = []
    for i in range(n):
        set_random_s0(env)
        env.reset()
        state_dist_episode = np.zeros(env.shape)
        for j in range(tmax):
            s = env.s
            state_dist_episode[s // env.shape[0]][s % env.shape[1]] += 1
            if(is_terminal(env, env.s)):
                break
            probs, state_value = model(s, s_star)
            action = probs.multinomial().data
            env.step(action[0][0])
        state_dist_list.append(state_dist_episode)
    
    state_dist = np.zeros(env.shape)
    
    for dist in state_dist_list:
        state_dist += dist
    
    return state_dist / n, state_dist_list  

In [12]:
agent_goal_based = Agent(state_dim, action_dim, "goal-based")
agent_simple = Agent(state_dim, action_dim, "simple")

optimizer_goal_based_agent = optim.RMSprop(agent_goal_based.parameters(), alpha=0.97, eps=1e-6, lr=lr_agent)
optimizer_simple_agent = optim.RMSprop(agent_simple.parameters(), alpha=0.97, eps=1e-6, lr=lr_agent)

In [13]:
import shutil
plots_dir = "plots_compare"
try:
    shutil.rmtree(plots_dir)
except:
    pass
try:
    os.mkdir(plots_dir)
except:
    pass

In [14]:
# import seaborn as sns;
# sns.set(color_codes=True)
# fig, ax = subplots(nrows=1, ncols=1, figsize=(5, 5))
# s_stars = [0, env.nS - 1, None]
# colors = sns.color_palette("Set1", n_colors=3, desat=.75)
# for s in range(1, env.nS - 1):
#     ax.set_xticks(np.arange(0, 6))
#     ax.set_yticks(np.arange(0, 6))
#     probs = Expert_global_optimal.get_action_probs(env, s)
#     draw_direction_probs(ax, env, s, probs)
#     ax.invert_yaxis()

# fig.tight_layout()
# #img = fig2img(fig)
# #writer.add_image('Action_Probs', np.array(img)[..., :3], 1)



# #plt.savefig(os.path.join(plots_dir, "action_prob_plot{}".format(0)))

In [15]:
# import seaborn as sns;
# sns.set(color_codes=True)
# fig, ax = subplots(nrows=1, ncols=3, figsize=(15, 5))
# s_stars = [0, env.nS - 1, None]
# colors = sns.color_palette("Set1", n_colors=3, desat=.75)
# for j, s_star in enumerate(s_stars):
#     for s in range(1, env.nS - 1):
#         if s is None:
#             agent = agent_simle
#         else:
#             agent = agent_goal_based
#         ax[j].set_xticks(np.arange(0, 6))
#         ax[j].set_yticks(np.arange(0, 6))
#         probs, state_value = agent(s, s_star)
#         draw_direction_probs(ax[j], env, s, probs[0].data, arrow_color=colors[j])
#         ax[j].set_title("S_star={}".format(str(s_star)), fontsize=16)
#         ax[j].invert_yaxis()

# fig.tight_layout()
# img = fig2img(fig)
# writer.add_image('Action_Probs', np.array(img)[..., :3], 1)



# #plt.savefig(os.path.join(plots_dir, "action_prob_plot{}".format(0)))

In [16]:
losses_history_goal_based = {
    "entropy":[],
    "value loss":[],
    "log loss":[]
}

losses_history_simple = {
    "entropy":[],
    "value loss":[],
    "log loss":[]
}

In [ ]:
each_goal_times = 1
times = 2000
start = 0

models = [agent_goal_based, agent_simple, Expert_global_optimal]
previous_rewards = [[] for i in range(len(models))]

for i in tnrange(times):
    agent_goal_based, rewards = train_agent(env, tmax, each_goal_times, agent_goal_based, optimizer_goal_based_agent, losses_history_goal_based, agent_type="goal-based")
    agent_simple, rewards = train_agent(env, tmax, each_goal_times, agent_simple, optimizer_simple_agent, losses_history_simple, agent_type="simple")
    
    fig, ax = subplots(nrows=1, ncols=1, figsize=(12, 6))
    models = [agent_goal_based, agent_simple, Expert_global_optimal]
    models_anotations = ["goal-based agent", "simple agent", "Expert global optimal"]
    previous_rewards = draw_reward_curves(writer, i, ax, env, models, models_anotations, tmax, previous_rewards, possible_s_stars)
    #fig.tight_layout()
    #plt.savefig(os.path.join(plots_dir, "rewards_curves{}".format(each_goal_times * (start + i + 1))))
    #plt.cla()
    #plt.clf()
    
    
    #actions_prob_plot
    sns.set(color_codes=True)
    fig, ax = subplots(nrows=1, ncols=3, figsize=(15, 5))
    s_stars = [0, env.nS - 1, None]
    colors = sns.color_palette("Set1", n_colors=3, desat=.75)
    for j, s_star in enumerate(s_stars):
        for s in range(1, env.nS - 1):
            if s is None:
                agent = agent_simle
            else:
                agent = agent_goal_based
            ax[j].set_xticks(np.arange(0, 6))
            ax[j].set_yticks(np.arange(0, 6))
            probs, state_value = agent(s, s_star)
            draw_direction_probs(ax[j], env, s, probs[0].data, arrow_color=colors[j])
            if s is None:
                ax[j].set_title("Simple agent", fontsize=16)
            else:
                ax[j].set_title("Goal-based agent, s_star={}".format(str(s_star)), fontsize=16)
            ax[j].invert_yaxis()

    fig.tight_layout()
    img = fig2img(fig)
    writer.add_image('Action_Probs', np.array(img)[..., :3], i)
    #plt.savefig(os.path.join(plots_dir, "action_prob_plot{}".format(each_goal_times * (start + i + 1))))
    #plt.cla()
    #plt.clf()
    
#     fig, ax = subplots(nrows=1, ncols=1, figsize=(15, 5))
#     ax.plot(losses_history_goal_based["entropy"][-100:], label="entropy term")
#     ax.plot(losses_history_goal_based["value loss"][-100:], label="value term")
#     ax.plot(losses_history_goal_based["log loss"][-100:], label="log loss term")
#     ax.legend(loc="best")
#     ax.set_xlabel("Episode")
#     ax.set_ylabel("Loss")
#     fig.tight_layout()
#     img = fig2img(fig)
    writer.add_scalars('loss terms',
                       {
                            "entropy term": losses_history_goal_based["entropy"][-1],
                            "value term": losses_history_goal_based["value loss"][-1],
                            "log loss term": losses_history_goal_based["log loss"][-1],
                       }, 
                       i)
    #plt.savefig(os.path.join(plots_dir, "loss terms"))
    #plt.cla()
    #plt.clf()
    
#     #reward_time_plot
#     fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))
#     ax[0].plot(rewards, label="Bob's reward")
#     ax[1].plot(time, label="Bob's time")
#     ax[2].hist(finish_states, color='c', alpha=0.5, label="Bob")

#     ax[0].set_xlabel("episode")
#     ax[1].set_xlabel("episode")
#     ax[1].set_ylabel("steps")
#     ax[0].set_ylabel("reward")
#     ax[2].set_xlabel("final state")
#     ax[2].set_ylabel("distribution")
#     ax[0].legend(loc="best")
#     ax[1].legend(loc="best")
#     ax[2].legend(loc="best")
#     fig.tight_layout()
#     plt.savefig(os.path.join(plots_dir, "reward_time_final{}".format(episodes_step * (start + i + 1))))
#     plt.clf()
#     #plt.savefig("plots_union_no_entropy_one_hot/reward_time_final{}".format(episodes_step * (start + i + 1)))
    
#     #final_states_heat_map
#     dense_Bob = np.zeros((5, 5))
#     for finish in finish_states:
#         dense_Bob[finish // 5][finish % 5] += 1

#     fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))

#     ax.imshow(dense_Bob, cmap='hot', interpolation='nearest')
#     ax.set_title("Bob final states")

#     plt.savefig(os.path.join(plots_dir, "final_states{}".format(episodes_step * (start + i + 1))))
#     plt.clf()
    
    fig, ax = subplots(nrows=1, ncols=3, figsize=(12, 5))
    for j in range(3):
        ax[j].set_xticks(np.arange(0, 6))
        ax[j].set_yticks(np.arange(0, 6))
        ax[j].invert_yaxis()

    for j, s_star in enumerate([0, 24, None]):
        if s_star is None:
            agent = agent_simple
        else:
            agent = agent_goal_based
        #print(agent.name)
        state_dist, state_dist_list = play_n_episodes(10, env, agent, s_star)   
        for s in range(1, env.nS - 1):
            value = agent(s, s_star)[1].data[0][0]
            #states_heat_map[int(s // np.sqrt(env.nS))][s % int(np.sqrt(env.nS))] = value
            draw_value_anotate(ax[j], env, s, value)
        ax[j].imshow(state_dist, cmap='hot', interpolation='nearest')
        if s is None:
            ax[j].set_title("Simple agent", fontsilze=16)
        else:
            ax[j].set_title("Goal-based agent, s_star={}".format(str(s_star)), fontsize=16)
    
    fig.tight_layout()
    img = fig2img(fig)
    writer.add_image('Value state distr', np.array(img)[..., :3], i)
    
    #plt.savefig(os.path.join(plots_dir, "value_state_distr{}".format(each_goal_times * (start + i + 1))))
    #plt.cla()
    #plt.clf()

/Users/andrejklimkin/Documents/Coursework/2017-2018/main/source/experts-learning/nets.py:53: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(action_scores), state_values


In [ ]:
board.wait(board_timeout)

In [ ]:
np.mean(get_policy_reward_estimation(env, agent_goal_based, 'agent', 1000, 40, s_star=[0, 24]))

In [ ]:
s = 1
agent_goal_based(s, 0)

In [ ]:
agent_simple(s)

In [ ]:
np.mean(get_policy_reward_estimation(env, agent_simple, 'agent', 1000, 40, s_star=[None]))

In [ ]:
agent_simple(s)

In [ ]:
probs = Variable(torch.FloatTensor([0.5, 0.5]))
m = torch.distributions.Categorical(probs)
m.log_prob(Variable(torch.LongTensor(1)))